# Preprocessing

Load World Cup 2018 data from StatsBomb JSON, filter to shots, and create the cleaned shot-level dataset with engineered features (x, y, distance, angle, etc.). Finally, save the result to CSV for use in the modeling notebook.

## Imports and configuration

In [144]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

pd.set_option('display.max_columns', None)


## Load matches and events for World Cup 2018


In [145]:
# Base path to StatsBomb open-data on your machine
BASE = Path(r"C:\\Users\\traik\\Desktop\\Final project data\\open-data-master\\data")

# World Cup 2018: competition_id=43, season_id=3
matches_file = BASE / "matches" / "43" / "3.json"

with open(matches_file, "r", encoding="utf-8") as f:
    matches_wc = json.load(f)

match_ids_wc = [m["match_id"] for m in matches_wc]

events_folder = BASE / "events"
all_events_wc = []

for mid in match_ids_wc:
    fp = events_folder / f"{mid}.json"
    with open(fp, "r", encoding="utf-8") as f:
        all_events_wc.extend(json.load(f))

df_wc = pd.json_normalize(all_events_wc, sep="_")
df_wc.head()


,id,index,period,timestamp,minute,second,possession,duration,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,team_id,team_name,tactics_formation,tactics_lineup,related_events,location,player_id,player_name,position_id,position_name,pass_recipient_id,pass_recipient_name,pass_length,pass_angle,pass_height_id,pass_height_name,pass_end_location,pass_type_id,pass_type_name,pass_body_part_id,pass_body_part_name,under_pressure,carry_end_location,pass_outcome_id,pass_outcome_name,counterpress,ball_receipt_outcome_id,ball_receipt_outcome_name,50_50_outcome_id,50_50_outcome_name,pass_cross,ball_recovery_recovery_failure,pass_switch,dribble_outcome_id,dribble_outcome_name,duel_type_id,duel_type_name,duel_outcome_id,duel_outcome_name,foul_committed_type_id,foul_committed_type_name,shot_statsbomb_xg,shot_end_location,shot_follows_dribble,shot_type_id,shot_type_name,shot_technique_id,shot_technique_name,shot_outcome_id,shot_outcome_name,shot_body_part_id,shot_body_part_name,shot_freeze_frame,goalkeeper_position_id,goalkeeper_position_name,goalkeeper_type_id,goalkeeper_type_name,goalkeeper_technique_id,goalkeeper_technique_name,goalkeeper_outcome_id,goalkeeper_outcome_name,goalkeeper_body_part_id,goalkeeper_body_part_name,interception_outcome_id,interception_outcome_name,pass_assisted_shot_id,pass_shot_assist,shot_key_pass_id,goalkeeper_end_location,clearance_aerial_won,foul_won_defensive,shot_open_goal,pass_aerial_won,foul_committed_advantage,foul_won_advantage,pass_backheel,dribble_nutmeg,pass_deflected,block_deflection,bad_behaviour_card_id,bad_behaviour_card_name,foul_committed_card_id,foul_committed_card_name,foul_committed_penalty,foul_won_penalty,block_offensive,substitution_outcome_id,substitution_outcome_name,substitution_replacement_id,substitution_replacement_name,shot_aerial_won,shot_first_time,pass_goal_assist,shot_deflected,dribble_overrun,block_save_block,foul_committed_offensive,injury_stoppage_in_chain,pass_technique_id,pass_technique_name,pass_through_ball,shot_one_on_one,pass_cut_back,ball_recovery_offensive,miscontrol_aerial_won,pass_miscommunication,shot_redirect
0,de3be98d-e227-475b-bd55-f57a6a89d308,1,1,00:00:00.000,0,0,1,0.000,35,Starting XI,769,Colombia,1,Regular Play,769,Colombia,433.0,"[{'player': {'id': 4276, 'name': 'David Ospina...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,f50ccda4-b768-4f07-9136-8f79fd17dac5,2,1,00:00:00.000,0,0,1,0.754,35,Starting XI,769,Colombia,1,Regular Play,768,England,352.0,"[{'player': {'id': 3468, 'name': 'Jordan Pickf...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b5e98805-0a22-4a5e-a306-7d40651a0f6e,3,1,00:00:00.000,0,0,1,9.320,18,Half Start,769,Colombia,1,Regular Play,768,England,NaN,NaN,[762b829f-5f24-4dd7-bfe2-da7e289838bb],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,762b829f-5f24-4dd7-bfe2-da7e289838bb,4,1,00:00:00.000,0,0,1,9.053,18,Half Start,769,

In [146]:
df_wc['id'].duplicated().sum()


0

## Filter to shots only

In [147]:
df_shots = df_wc[df_wc["type_name"] == "Shot"].copy()
df_shots.shape


(1706, 121)

### Filter out penalty shootouts

In [148]:
df_penalty_shootouts = df_wc[df_wc["period"] == 5]
display(df_penalty_shootouts.shape)
df_penalty_shootouts[["type_name", "shot_type_name", "location", "period"]].head(15)


(94, 121)

,type_name,shot_type_name,location,period
3995,Half Start,NaN,NaN,5
3996,Half Start,NaN,NaN,5
3997,Shot,Penalty,"[109.0, 41.0]",5
3998,Goal Keeper,NaN,"[1.0, 40.0]",5
3999,Shot,Penalty,"[109.0, 41.0]",5
4000,Goal Keeper,NaN,"[1.0, 40.0]",5
4001,Shot,Penalty,"[109.0, 41.0]",5
4002,Goal Keeper,NaN,"[1.0, 40.0]",5
4003,Shot,Penalty,"[109.0, 41.0]",5
4004,Goal Keeper,NaN,"[1.0, 40.0]",5


In [149]:
shots = df_shots[df_shots["period"] != 5]


## Select relevant columns


In [150]:
cols = [
    "location",
    "counterpress",
    "shot_statsbomb_xg",
    "shot_end_location",
    "shot_type_id",
    "shot_technique_id",
    "shot_outcome_id",
    "shot_body_part_id",
    "shot_open_goal",
    "shot_first_time",
    "shot_one_on_one",
    "shot_aerial_won",
]

shots = shots[cols]
shots.head()


,location,counterpress,shot_statsbomb_xg,shot_end_location,shot_type_id,shot_technique_id,shot_outcome_id,shot_body_part_id,shot_open_goal,shot_first_time,shot_one_on_one,shot_aerial_won
239,"[115.0, 18.0]",NaN,0.009816,"[120.0, 42.2, 2.0]",62.0,93.0,100.0,40.0,NaN,NaN,NaN,NaN
270,"[112.0, 54.0]",NaN,0.038204,"[113.0, 53.0]",87.0,93.0,96.0,40.0,NaN,NaN,NaN,NaN
439,"[98.0, 37.0]",NaN,0.045128,"[105.0, 37.0]",87.0,93.0,96.0,40.0,NaN,NaN,NaN,NaN
548,"[119.0, 36.0]",NaN,0.625074,"[120.0, 40.5, 3.1]",87.0,93.0,98.0,37.0,True,NaN,NaN,NaN
815,"[97.0, 56.0]",NaN,0.021760,"[100.0, 54.0]",87.0,93.0,96.0,40.0,NaN,NaN,NaN,NaN


## Drop unreliable columns



In [151]:
shots.isna().sum()

location                0
counterpress         1667
shot_statsbomb_xg       0
shot_end_location       0
shot_type_id            0
shot_technique_id       0
shot_outcome_id         0
shot_body_part_id       0
shot_open_goal       1650
shot_first_time      1310
shot_one_on_one      1612
shot_aerial_won      1543
dtype: int64

In [152]:
# Drop counterpress

shots.drop(["counterpress"], axis=1, inplace=True)

shots.isna().sum()


location                0
shot_statsbomb_xg       0
shot_end_location       0
shot_type_id            0
shot_technique_id       0
shot_outcome_id         0
shot_body_part_id       0
shot_open_goal       1650
shot_first_time      1310
shot_one_on_one      1612
shot_aerial_won      1543
dtype: int64

In [153]:
# Drop shot_end_location as execution of shot should not play a role in measuring the quality of shot.

shots.drop(["shot_end_location"], axis=1, inplace=True)

## Clean boolean shot flags

`shot_open_goal`, `shot_first_time`, `shot_one_on_one`, `shot_aerial_won` are mapped from `NaN` → 0 and cast to integer 0/1.

In [154]:
binary_cols = [
    "shot_open_goal",
    "shot_first_time",
    "shot_one_on_one",
    "shot_aerial_won",
]

for col in binary_cols:
    shots[col] = shots[col].fillna(0).astype(int)

shots[binary_cols].head()


,shot_open_goal,shot_first_time,shot_one_on_one,shot_aerial_won
239,0,0,0,0
270,0,0,0,0
439,0,0,0,0
548,1,0,0,0
815,0,0,0,0


## Extract shot location (x, y)

We convert the StatsBomb `location` list `[x, y]` into separate numeric `x` and `y` columns, then drop the original `location` column.

In [155]:
shots["x"] = shots["location"].apply(lambda loc: loc[0])
shots["y"] = shots["location"].apply(lambda loc: loc[1])

shots.drop("location", axis=1, inplace=True)
shots.head()


,shot_statsbomb_xg,shot_type_id,shot_technique_id,shot_outcome_id,shot_body_part_id,shot_open_goal,shot_first_time,shot_one_on_one,shot_aerial_won,x,y
239,0.009816,62.0,93.0,100.0,40.0,0,0,0,0,115.0,18.0
270,0.038204,87.0,93.0,96.0,40.0,0,0,0,0,112.0,54.0
439,0.045128,87.0,93.0,96.0,40.0,0,0,0,0,98.0,37.0
548,0.625074,87.0,93.0,98.0,37.0,1,0,0,0,119.0,36.0
815,0.021760,87.0,93.0,96.0,40.0,0,0,0,0,97.0,56.0


## Create goal label `is_goal`

We use your mapping: `shot_outcome_id == 97` corresponds to goals in this dataset.

In [156]:
shots["is_goal"] = (shots["shot_outcome_id"] == 97).astype(int)
shots["is_goal"].value_counts()


is_goal
0    1510
1     157
Name: count, dtype: int64

In [157]:
# Drop the outcome_id now that we have the label
shots.drop("shot_outcome_id", axis=1, inplace=True)
shots.head()


,shot_statsbomb_xg,shot_type_id,shot_technique_id,shot_body_part_id,shot_open_goal,shot_first_time,shot_one_on_one,shot_aerial_won,x,y,is_goal
239,0.009816,62.0,93.0,40.0,0,0,0,0,115.0,18.0,0
270,0.038204,87.0,93.0,40.0,0,0,0,0,112.0,54.0,0
439,0.045128,87.0,93.0,40.0,0,0,0,0,98.0,37.0,0
548,0.625074,87.0,93.0,37.0,1,0,0,0,119.0,36.0,0
815,0.021760,87.0,93.0,40.0,0,0,0,0,97.0,56.0,0


## Add geometry features: distance and angle

We compute:
- `distance`: distance from shot location to the centre of the goal
- `angle`: angle between lines from the shot location to the two goalposts

Pitch and goal coordinates follow the StatsBomb convention: x ∈ [0,120], y ∈ [0,80], goal centered at (120, 40).

In [158]:
# Goal coordinates (StatsBomb pitch)
goal_x = 120
goal_y = 40
left_post_y = 36.8
right_post_y = 43.2

# Distance to goal centre
shots["distance"] = np.sqrt((goal_x - shots["x"])**2 + (goal_y - shots["y"])**2)

def calc_angle(row):
    x = row["x"]
    y = row["y"]
    angle_left = np.arctan2(left_post_y - y, goal_x - x)
    angle_right = np.arctan2(right_post_y - y, goal_x - x)
    return abs(angle_right - angle_left)



## Create final featured shots dataframe

We drop `shot_end_location` so the features only describe the chance **before** the shot outcome. This final dataframe `shots_featured` will be saved to CSV and used in the modeling notebook.

In [159]:
shots_featured = shots.copy()

shots_featured["angle"] = shots.apply(calc_angle, axis=1)
shots_featured[["x", "y", "distance", "angle"]].head()


shots_featured.head()


,shot_statsbomb_xg,shot_type_id,shot_technique_id,shot_body_part_id,shot_open_goal,shot_first_time,shot_one_on_one,shot_aerial_won,x,y,is_goal,distance,angle
239,0.009816,62.0,93.0,40.0,0,0,0,0,115.0,18.0,0,22.561028,0.064071
270,0.038204,87.0,93.0,40.0,0,0,0,0,112.0,54.0,0,16.124515,0.202196
439,0.045128,87.0,93.0,40.0,0,0,0,0,98.0,37.0,0,22.203603,0.283785
548,0.625074,87.0,93.0,37.0,1,0,0,0,119.0,36.0,0,4.123106,0.758049
815,0.021760,87.0,93.0,40.0,0,0,0,0,97.0,56.0,0,28.017851,0.187756


### Checking for duplicate columns

In [160]:
shots_featured.duplicated().sum()

22

In [161]:
dupes = shots_featured[shots_featured.duplicated(keep=False)]
dupes.head(60)



,shot_statsbomb_xg,shot_type_id,shot_technique_id,shot_body_part_id,shot_open_goal,shot_first_time,shot_one_on_one,shot_aerial_won,x,y,is_goal,distance,angle
1972,0.7835,88.0,93.0,40.0,0,0,0,0,108.0,40.0,1,12.000000,0.521205
13367,0.7835,88.0,93.0,40.0,0,0,0,0,108.0,40.0,0,12.000000,0.521205
14420,0.7835,88.0,93.0,40.0,0,0,0,0,109.0,41.0,1,11.045361,0.562130
22589,0.7835,88.0,93.0,40.0,0,0,0,0,109.0,41.0,1,11.045361,0.562130
23240,0.7835,88.0,93.0,40.0,0,0,0,0,109.0,41.0,1,11.045361,0.562130
60457,0.7835,88.0,93.0,40.0,0,0,0,0,109.0,41.0,1,11.045361,0.562130
61587,0.7835,88.0,93.0,40.0,0,0,0,0,109.0,41.0,1,11.045361,0.562130
74316,0.7835,88.0,93.0,40.0,0,0,0,0,108.0,40.0,1,12.000000,0.521205
77105,0.7835,88.0,93.0,38.0,0,0,0,0,108.0,40.0,1,12.000000,0.521205
90799,0.7835,88.0,93.0,40.0,0,0,0,0,108.0,40.0,0,12.000000,0.521205


In [162]:
# duplicates are mostly penalty shots. we dont drop them.


In [163]:
shots_featured.iloc[32:45,:]

,shot_statsbomb_xg,shot_type_id,shot_technique_id,shot_body_part_id,shot_open_goal,shot_first_time,shot_one_on_one,shot_aerial_won,x,y,is_goal,distance,angle
4454,0.257208,87.0,93.0,40.0,0,0,0,0,117.0,40.0,0,3.000000,1.635290
4540,0.006950,87.0,93.0,40.0,0,0,0,0,91.0,20.0,0,35.227830,0.149673
4604,0.173503,87.0,93.0,37.0,0,0,0,0,112.0,42.0,0,8.246211,0.725265
5209,0.037858,87.0,93.0,40.0,0,0,0,0,109.0,55.0,0,18.601075,0.206680
5215,0.068467,87.0,93.0,40.0,0,0,0,0,107.0,33.0,0,14.764823,0.380905
5228,0.019955,87.0,93.0,40.0,0,0,0,0,91.0,49.0,0,30.364453,0.200819
5497,0.085507,87.0,93.0,37.0,0,0,0,0,113.0,40.0,0,7.000000,0.857556
5511,0.024078,87.0,93.0,38.0,0,0,0,0,90.0,41.0,0,30.016662,0.212298
5516,0.101480,87.0,90.0,37.0,0,0,0,0,111.0,43.0,0,9.486833,0.625448
5559,0.071149,87.0,93.0,37.0,0,0,0,1,109.0,35.0,0,12.083046,0.478387


In [164]:
shots_featured.shape

(1667, 13)

In [165]:
shots_featured.is_goal.value_counts()

is_goal
0    1510
1     157
Name: count, dtype: int64

## Save preprocessed shots to CSV

In [166]:
output_path = Path("shots_featured_wc2018.csv")
shots_featured.to_csv(output_path, index=False)
print(f"Saved preprocessed shots to {output_path.resolve()}")


Saved preprocessed shots to C:\Users\traik\Desktop\ML_Bootcamp\Final_Project\Final_Project_Ironhack\shots_featured_wc2018.csv


# Loading Euro 2020 Data to test models on

In [167]:


# EURO 2020
matches_file_euro = BASE / "matches" / "55" / "43.json"
with open(matches_file_euro, "r", encoding="utf-8") as f:
    matches_euro = json.load(f)

match_ids_euro = [m["match_id"] for m in matches_euro]

# Load events
events_folder = BASE / "events"
all_events_euro = []

for mid in match_ids_euro:
    fp = events_folder / f"{mid}.json"
    with open(fp, "r", encoding="utf-8") as f:
        all_events_euro.extend(json.load(f))

df_euro = pd.json_normalize(all_events_euro, sep="_")
df_euro.head()

,id,index,period,timestamp,minute,second,possession,duration,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,team_id,team_name,tactics_formation,tactics_lineup,related_events,location,player_id,player_name,position_id,position_name,pass_recipient_id,pass_recipient_name,pass_length,pass_angle,pass_height_id,pass_height_name,pass_end_location,pass_body_part_id,pass_body_part_name,pass_type_id,pass_type_name,carry_end_location,under_pressure,pass_outcome_id,pass_outcome_name,ball_receipt_outcome_id,ball_receipt_outcome_name,duel_type_id,duel_type_name,clearance_head,clearance_body_part_id,clearance_body_part_name,clearance_aerial_won,duel_outcome_id,duel_outcome_name,interception_outcome_id,interception_outcome_name,shot_statsbomb_xg,shot_end_location,shot_technique_id,shot_technique_name,shot_body_part_id,shot_body_part_name,shot_type_id,shot_type_name,shot_outcome_id,shot_outcome_name,shot_freeze_frame,goalkeeper_end_location,goalkeeper_position_id,goalkeeper_position_name,goalkeeper_type_id,goalkeeper_type_name,off_camera,pass_cross,clearance_left_foot,goalkeeper_outcome_id,goalkeeper_outcome_name,counterpress,out,miscontrol_aerial_won,ball_recovery_recovery_failure,pass_switch,pass_assisted_shot_id,pass_shot_assist,pass_outswinging,pass_technique_id,pass_technique_name,shot_key_pass_id,shot_first_time,dribble_outcome_id,dribble_outcome_name,pass_aerial_won,clearance_right_foot,foul_committed_type_id,foul_committed_type_name,foul_won_defensive,pass_inswinging,substitution_outcome_id,substitution_outcome_name,substitution_replacement_id,substitution_replacement_name,goalkeeper_technique_id,goalkeeper_technique_name,goalkeeper_body_part_id,goalkeeper_body_part_name,shot_aerial_won,50_50_outcome_id,50_50_outcome_name,dribble_overrun,ball_recovery_offensive,foul_committed_advantage,foul_won_advantage,block_deflection,pass_through_ball,foul_committed_card_id,foul_committed_card_name,pass_goal_assist,block_offensive,pass_cut_back,pass_deflected,shot_one_on_one,foul_committed_offensive,injury_stoppage_in_chain,pass_straight,pass_no_touch,pass_miscommunication,goalkeeper_punched_out,foul_committed_penalty,foul_won_penalty,block_save_block,shot_redirect,dribble_nutmeg,shot_deflected,bad_behaviour_card_id,bad_behaviour_card_name,clearance_other,shot_open_goal,shot_saved_to_post,goalkeeper_shot_saved_to_post,shot_saved_off_target,goalkeeper_shot_saved_off_target,goalkeeper_lost_in_play,goalkeeper_success_in_play,dribble_no_touch,goalkeeper_penalty_saved_to_post,shot_follows_dribble,player_off_permanent
0,2e4b1b7f-e67e-4bef-8cf1-2eab2ab81af2,1,1,00:00:00.000,0,0,1,0.000000,35,Starting XI,773,Switzerland,1,Regular Play,773,Switzerland,4231.0,"[{'player': {'id': 5550, 'name': 'Yann Sommer'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,f73c3f38-8c45-4802-8cb9-5a87234f385b,2,1,00:00:00.000,0,0,1,0.000000,35,Starting XI,773,Switzerland,1,Regular Play,772,Spain,433.0,"[{'player': {'id': 11748, 'name': 'Unai Simón ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9098375d-3f07-4241-bdc9-3602a2e5c754,3,1,00:00:0

### checking for duplicates in original data frame

In [168]:
df_euro["id"].duplicated().sum()

0

# Filter Data Frame to type shot

In [169]:
df_shots_euro = df_euro[df_euro["type_name"] == "Shot"].copy()
df_shots_euro.shape

(1289, 142)

remove penalty shootouts

In [170]:
df_penalty_shootouts_euro = df_euro[df_euro["period"] == 5]
display(df_penalty_shootouts_euro.shape)
df_penalty_shootouts_euro[["type_name", "shot_type_name", "location", "period"]].head(15)


(92, 142)

,type_name,shot_type_name,location,period
4968,Half Start,NaN,NaN,5
4969,Half Start,NaN,NaN,5
4970,Shot,Penalty,"[108.0, 40.0]",5
4971,Goal Keeper,NaN,"[1.0, 40.0]",5
4972,Shot,Penalty,"[108.0, 40.0]",5
4973,Goal Keeper,NaN,"[1.0, 40.0]",5
4974,Shot,Penalty,"[108.0, 40.0]",5
4975,Goal Keeper,NaN,"[1.0, 40.0]",5
4976,Shot,Penalty,"[108.0, 40.0]",5
4977,Goal Keeper,NaN,"[1.0, 40.0]",5


In [171]:
shots_euro = df_shots_euro[df_shots_euro["period"] != 5]
shots_euro.shape


(1251, 142)

Euro data set will only be used for evaluation. Thus, the same feature and pre-processing steps will be maintained and applied as before.

In [172]:
cols = [
    "location",
    "counterpress",
    "shot_statsbomb_xg",
    "shot_end_location",
    "shot_type_id",
    "shot_technique_id",
    "shot_outcome_id",
    "shot_body_part_id",
    "shot_open_goal",
    "shot_first_time",
    "shot_one_on_one",
    "shot_aerial_won",
]

shots_euro = shots_euro[cols]
display(shots_euro.head())
shots_euro.shape


,location,counterpress,shot_statsbomb_xg,shot_end_location,shot_type_id,shot_technique_id,shot_outcome_id,shot_body_part_id,shot_open_goal,shot_first_time,shot_one_on_one,shot_aerial_won
72,"[91.9, 39.0]",NaN,0.030169,"[99.7, 40.0]",87.0,93.0,96.0,38.0,NaN,NaN,NaN,NaN
400,"[96.5, 29.7]",NaN,0.021315,"[110.2, 37.8]",87.0,91.0,96.0,38.0,NaN,True,NaN,NaN
748,"[97.8, 25.9]",NaN,0.047888,"[120.0, 38.3, 3.5]",62.0,93.0,98.0,40.0,NaN,NaN,NaN,NaN
1016,"[112.0, 40.7]",NaN,0.100356,"[118.8, 38.7, 2.2]",87.0,93.0,100.0,37.0,NaN,NaN,NaN,NaN
1449,"[99.7, 58.6]",NaN,0.008758,"[100.7, 57.7]",87.0,93.0,96.0,38.0,NaN,NaN,NaN,NaN


(1251, 12)

In [173]:
display(shots_euro.isna().sum())
shots_euro.shape

location                0
counterpress         1251
shot_statsbomb_xg       0
shot_end_location       0
shot_type_id            0
shot_technique_id       0
shot_outcome_id         0
shot_body_part_id       0
shot_open_goal       1236
shot_first_time       857
shot_one_on_one      1206
shot_aerial_won      1117
dtype: int64

(1251, 12)

In [174]:
# Drop counterpress

shots_euro.drop(["counterpress"], axis=1, inplace=True)

shots_euro.isna().sum()


location                0
shot_statsbomb_xg       0
shot_end_location       0
shot_type_id            0
shot_technique_id       0
shot_outcome_id         0
shot_body_part_id       0
shot_open_goal       1236
shot_first_time       857
shot_one_on_one      1206
shot_aerial_won      1117
dtype: int64

In [175]:
# Drop Shot_end_location
shots_euro.drop(["shot_end_location"], axis=1, inplace=True)


In [176]:
display(shots_euro.shape)
shots.shape


(1251, 10)

(1667, 12)

In [177]:
# map false to 0 for binary columns. apply astype(int)

binary_cols = [
    "shot_open_goal",
    "shot_first_time",
    "shot_one_on_one",
    "shot_aerial_won",
]

for col in binary_cols:
    shots_euro[col] = shots_euro[col].fillna(0).astype(int)

display(shots_euro[binary_cols].head())

shots_euro.isna().sum()



,shot_open_goal,shot_first_time,shot_one_on_one,shot_aerial_won
72,0,0,0,0
400,0,1,0,0
748,0,0,0,0
1016,0,0,0,0
1449,0,0,0,0


location             0
shot_statsbomb_xg    0
shot_type_id         0
shot_technique_id    0
shot_outcome_id      0
shot_body_part_id    0
shot_open_goal       0
shot_first_time      0
shot_one_on_one      0
shot_aerial_won      0
dtype: int64

In [178]:
# Extracting shot locations

shots_euro["x"] = shots_euro["location"].apply(lambda loc: loc[0])
shots_euro["y"] = shots_euro["location"].apply(lambda loc: loc[1])

shots_euro.drop("location", axis=1, inplace=True)
display(shots_euro.head())
shots_euro.shape


,shot_statsbomb_xg,shot_type_id,shot_technique_id,shot_outcome_id,shot_body_part_id,shot_open_goal,shot_first_time,shot_one_on_one,shot_aerial_won,x,y
72,0.030169,87.0,93.0,96.0,38.0,0,0,0,0,91.9,39.0
400,0.021315,87.0,91.0,96.0,38.0,0,1,0,0,96.5,29.7
748,0.047888,62.0,93.0,98.0,40.0,0,0,0,0,97.8,25.9
1016,0.100356,87.0,93.0,100.0,37.0,0,0,0,0,112.0,40.7
1449,0.008758,87.0,93.0,96.0,38.0,0,0,0,0,99.7,58.6


(1251, 11)

In [179]:
# Replacing shou_outcome with is_goal feature

shots_euro["is_goal"] = (shots_euro["shot_outcome_id"] == 97).astype(int)
shots_euro["is_goal"].value_counts()


is_goal
0    1120
1     131
Name: count, dtype: int64

In [180]:
# Drop the outcome_id now that we have the label
shots_euro.drop("shot_outcome_id", axis=1, inplace=True)
display(shots_euro.head())
shots_euro.shape


,shot_statsbomb_xg,shot_type_id,shot_technique_id,shot_body_part_id,shot_open_goal,shot_first_time,shot_one_on_one,shot_aerial_won,x,y,is_goal
72,0.030169,87.0,93.0,38.0,0,0,0,0,91.9,39.0,0
400,0.021315,87.0,91.0,38.0,0,1,0,0,96.5,29.7,0
748,0.047888,62.0,93.0,40.0,0,0,0,0,97.8,25.9,0
1016,0.100356,87.0,93.0,37.0,0,0,0,0,112.0,40.7,0
1449,0.008758,87.0,93.0,38.0,0,0,0,0,99.7,58.6,0


(1251, 11)

In [181]:
# Adding geometry features

# Distance to goal centre

shots_euro["distance"] = np.sqrt((goal_x - shots_euro["x"])**2 + (goal_y - shots_euro["y"])**2)

shots_euro["angle"] = shots_euro.apply(calc_angle,axis=1)

In [182]:
shots_euro.head()

,shot_statsbomb_xg,shot_type_id,shot_technique_id,shot_body_part_id,shot_open_goal,shot_first_time,shot_one_on_one,shot_aerial_won,x,y,is_goal,distance,angle
72,0.030169,87.0,93.0,38.0,0,0,0,0,91.9,39.0,0,28.117788,0.226500
400,0.021315,87.0,91.0,38.0,0,1,0,0,96.5,29.7,0,25.658137,0.228027
748,0.047888,62.0,93.0,40.0,0,0,0,0,97.8,25.9,0,26.299240,0.205564
1016,0.100356,87.0,93.0,37.0,0,0,0,0,112.0,40.7,0,8.030567,0.756483
1449,0.008758,87.0,93.0,38.0,0,0,0,0,99.7,58.6,0,27.532708,0.172017


In [183]:
display(shots_featured.shape)
display(shots.shape)
shots_euro.shape

(1667, 13)

(1667, 12)

(1251, 13)

In [184]:
display(shots_featured.columns)
display(shots.columns)
display(shots_euro.columns)

Index(['shot_statsbomb_xg', 'shot_type_id', 'shot_technique_id',
       'shot_body_part_id', 'shot_open_goal', 'shot_first_time',
       'shot_one_on_one', 'shot_aerial_won', 'x', 'y', 'is_goal', 'distance',
       'angle'],
      dtype='object')

Index(['shot_statsbomb_xg', 'shot_type_id', 'shot_technique_id',
       'shot_body_part_id', 'shot_open_goal', 'shot_first_time',
       'shot_one_on_one', 'shot_aerial_won', 'x', 'y', 'is_goal', 'distance'],
      dtype='object')

Index(['shot_statsbomb_xg', 'shot_type_id', 'shot_technique_id',
       'shot_body_part_id', 'shot_open_goal', 'shot_first_time',
       'shot_one_on_one', 'shot_aerial_won', 'x', 'y', 'is_goal', 'distance',
       'angle'],
      dtype='object')

In [185]:
# checking for duplicates before saving file

shots_euro.duplicated().sum()

6

In [186]:
shots_euro.duplicated

<bound method DataFrame.duplicated of         shot_statsbomb_xg  shot_type_id  shot_technique_id  shot_body_part_id  \
72               0.030169          87.0               93.0               38.0   
400              0.021315          87.0               91.0               38.0   
748              0.047888          62.0               93.0               40.0   
1016             0.100356          87.0               93.0               37.0   
1449             0.008758          87.0               93.0               38.0   
...                   ...           ...                ...                ...   
192306           0.030229          87.0               93.0               38.0   
192352           0.043607          87.0               93.0               40.0   
192386           0.025420          87.0               91.0               40.0   
192531           0.058556          87.0               91.0               40.0   
192561           0.127872          87.0               93.0             

These are not duplicates but penalty shots taken in games. Penatly shootouts are already removed

# Save Euro 2020 Data to csv file


In [187]:
# save data frame to csv file

output_path = Path("shots_featured_eu2020.csv")
shots_euro.to_csv(output_path, index=False)
print(f"Saved Euro preprocessed shots to {output_path.resolve()}")


Saved Euro preprocessed shots to C:\Users\traik\Desktop\ML_Bootcamp\Final_Project\Final_Project_Ironhack\shots_featured_eu2020.csv
